# Ejercicio 8

Utilizando el archivo Iris.csv que contiene información referida a la longitud y al ancho de sépalos y pétalos de tres especies de flores: iris setosa, iris versicolor e iris virginica.


# Inciso A

Entrenar un multiperceptrón que aprenda a clasificar las 3 clases de flores.


# Inciso B

Utilice Python para calcular la matriz de confusión y calcule de forma manual las métricas de precision, recall, accuracy y f1-score.  
Luego utilice la función classification_report de SciKit-Learn para comparar los resultados.
